In [1]:
%%python2
from cpr.Nivel import Nivel
from cpr.SqlDb import SqlDb
import cpr.information as info

Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
  File "/usr/local/lib/python2.7/dist-packages/cpr/Nivel.py", line 14, in <module>
    from mpl_toolkits.basemap import Basemap
ImportError: No module named basemap


In [12]:
def duplicate_existing_table(self,table_name):
    '''
    inserts data into SQL table from list of fields and values
    Parameters
    ----------
    table_name   = SQL db table name
    Returns
    -------
    Sql sentence,str
    '''
    df = self.read_sql('describe %s'%table_name)
    df['Null'][df['Null']=='NO'] = 'NOT NULL'
    df['Null'][df['Null']=='YES'] = 'NULL'
    sentence = 'CREATE TABLE %s ('%table_name
    if df[df['Extra']=='auto_increment'].empty:
        pk = None
    else:
        pk = df[df['Extra']=='auto_increment']['Field'].values[0]
    for id,serie in df.iterrows():
        if (serie.Default=='0') or (serie.Default is None):
            row = '%s %s %s'%(serie.Field,serie.Type,serie.Null)
        else:
            if (serie.Default == 'CURRENT_TIMESTAMP'):
                serie.Default = "DEFAULT %s"%serie.Default
            elif serie.Default == '0000-00-00':
                serie.Default = "DEFAULT '1000-01-01 00:00:00'"
            else:
                serie.Default = "DEFAULT '%s'"%serie.Default
            row = '%s %s %s %s'%(serie.Field,serie.Type,serie.Null,serie.Default)
        if serie.Extra:
            row += ' %s,'%serie.Extra
        else:
            row += ',' 
        sentence+=row
    if pk:
        sentence +='PRIMARY KEY (%s)'%pk
    else:
        sentence = sentence[:-1]
    sentence +=');'
    return sentence

def siata_remote_data_to_transfer(start,end):
    remote = cpr.Nivel(**cpr.info.REMOTE)
    codigos_str = '('+str(list(self.infost.index)).strip('[]')+')'
    df = remote.read_sql('SELECT * FROM datos WHERE cliente in %s and %s'%(codigos_str,self.fecha_hora_query(start,end)))
    return df

def default_values(self,table_name):
    describe_table = self.read_sql('describe %s'%table_name)
    not_null = describe_table['Default'].notnull()
    default_values = describe_table[['Field','Default','Type']][not_null].set_index('Field')[['Default','Type']]
    return default_values

def filter_data_to_update(table_name,path):
    default = default_values(self,table_name)
    if default[default['Type']=='time'].empty:
        time_fields = None
    else:
        time_fields = default[default['Type']=='time']
    for index,s in time_fields.iterrows():
        df = pd.read_csv(path,index_col=0)
        df[index] = df[index].apply(lambda x:str(x)[6:15])
    return df.applymap(lambda x:str(x))

def insert_in_datos(path):
    inicia = datetime.datetime.now()
    table_name = 'datos'
    df = filter_data_to_update(table_name,path)
    query = 'INSERT INTO %s '%table_name+'('+str(list(df.columns)).strip('[]').replace("'",'')+') VALUES '
    for id,s in df.iterrows():
        query+=('('+str(list(s.values)).strip('[]'))+'), '
    query = query[:-2]+' ON DUPLICATE KEY UPDATE '
    describe = self.read_sql('describe %s;'%table_name)
    not_primary_keys = describe[describe['Key']!='PRI']
    if not_primary_keys.empty:
        not_primary_keys = describe.Field.values
    else:
        not_primary_keys = not_primary_keys.Field.values
    for key in not_primary_keys:
        query+=('%s = VALUES(%s), '%(key,key))
    query=query[:-2]
    finaliza = datetime.datetime.now()
    self.user = 'root'
    self.passwd = 'chan104jak'
    self.execute_sql('SET GLOBAL max_allowed_packet=1073741824;')
    self.execute_sql(query)
    print(finaliza - inicia)
    
#os.system("scp -r mcano@siata.gov.co:data_migration/* ../../cprweb/src/media/data_migration/")

self = SqlDb(**info.REMOTE)

folder_path = '/home/alexa/Repositorios/cprweb/src/media/weekly_data/'
files = os.listdir(folder_path)



In [ ]:
for path in files:
    insert_in_datos(folder_path+path)

In [10]:
level = Nivel(codigo=99,**info.LOCAL).level('2018-03-01',datetime.datetime.now())

In [13]:
self = SqlDb(**info.REMOTE)
folder_path = '/home/alexa/Repositorios/cprweb/src/media/weekly_data/'
files = os.listdir(folder_path)

april = []
for path in files:
    if path[:20] == 'weekly_level_2018-04':
        april.append(path)

date_offset = pd.to_datetime(level.dropna().index[-1])
date = date_offset-pd.to_datetime(pd.read_csv(folder_path + april[0],index_col=0)['fecha'][0])
for path in april:
    raw_data = pd.read_csv(folder_path + path,index_col=0)
    raw_data['fecha'] = (pd.to_datetime(raw_data['fecha']) + datetime.timedelta(days=date.days)).apply(lambda x:x.strftime('%Y-%m-%d'))
    raw_data.to_csv(folder_path+'future_'+path)
    insert_in_datos(folder_path+'future_'+path)

0:00:40.495675
0:00:39.337454
0:00:40.418218
0:00:40.114404
0:00:39.968619
